In [ ]:
!pip install transformers==4.40.1 accelerate==0.30.0 bitsandbytes==0.43.1 datasets==2.19.0 vllm==0.4.1 openai==1.25.1 -qqq

# 8.4절 실습: LLM 서빙 프레임워크

## 예제 8.1. 실습에 사용할 데이터셋 준비

In [ ]:
import torch
from datasets import load_dataset

def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
{ddl}

### Question:
{question}

### SQL:
{query}"""
    return prompt

dataset = load_dataset("shangrilar/ko_text2sql", "origin")['test']
dataset = dataset.to_pandas()

for idx, row in dataset.iterrows():
  prompt = make_prompt(row['context'], row['question'])
  dataset.loc[idx, 'prompt'] = prompt

## 예제 8.2. 모델과 토크나이저를 불러와 추론 파이프라인 준비

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "shangrilar/yi-ko-6b-text2sql"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_id)
hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

## 예제 8.3. 배치 크기에 따른 추론 시간 확인

In [ ]:
import time
for batch_size in [1, 2, 4, 8, 16, 32]:
  start_time = time.time()
  hf_pipeline(dataset['prompt'].tolist(), max_new_tokens=128, batch_size=batch_size)
  print(f'{batch_size}: {time.time() - start_time}')

## 예제 8.4. vLLM 모델 불러오기

In [ ]:
from vllm import LLM, SamplingParams

model_id = "shangrilar/yi-ko-6b-text2sql"
llm = LLM(model=model_id, dtype=torch.float16, max_model_len=1024)

## 예제 8.5. vLLM을 활용한 오프라인 추론 시간 측정

In [ ]:
import time

for max_num_seqs in [1, 2, 4, 8, 16, 32]:
  start_time = time.time()
  llm.llm_engine.scheduler_config.max_num_seqs = max_num_seqs
  sampling_params = SamplingParams(temperature=1, top_p=1, max_tokens=128)
  outputs = llm.generate(dataset['prompt'].tolist(), sampling_params)
  print(f'{max_num_seqs}: {time.time() - start_time}')

## 예제 8.6. 온라인 서빙을 위한 vLLM API 서버 실행

In [ ]:
!python -m vllm.entrypoints.openai.api_server \
--model shangrilar/yi-ko-6b-text2sql --host 127.0.0.1 --port 8888 --max-model-len 1024

## 예제 8.7. 백그라운드에서 vLLM API 서버 실행하기

In [ ]:
!nohup python -m vllm.entrypoints.openai.api_server \
--model shangrilar/yi-ko-6b-text2sql --host 127.0.0.1 --port 8888 --max-model-len 512 &

## 예제 8.8. API 서버 실행 확인

In [ ]:
!curl http://localhost:8888/v1/models

## 예제 8.9. API 요청

In [ ]:
import json

json_data = json.dumps(
    {"model": "shangrilar/yi-ko-6b-text2sql",
      "prompt": dataset.loc[0, "prompt"],
      "max_tokens": 128,
      "temperature": 1}
    )

!curl http://localhost:8888/v1/completions \
    -H "Content-Type: application/json" \
    -d '{json_data}'

## 예제 8.10. OpenAI 클라이언트를 사용한 API 요청

In [ ]:
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8888/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
completion = client.completions.create(model="shangrilar/yi-ko-6b-text2sql",
                                 prompt=dataset.loc[0, 'prompt'], max_tokens=128)
print("생성 결과:", completion.choices[0].text)